<a href="https://colab.research.google.com/github/Hydrometeorological-Remote-Sensing/cropmapping/blob/sw_ww_test/predict_pytorch_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q segmentation_models_pytorch 
!pip install -q affine
!pip install -q rioxarray 

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
!cp /content/drive/MyDrive/RDA/model_and_env/data_loader.py data_loader.py
!cp /content/drive/MyDrive/RDA/model_and_env/unet_pytorch.py unet_pytorch.py

In [ ]:
import datetime
import segmentation_models_pytorch as smp
import numpy as np
import torch
from glob import glob
import torch.utils.data
from data_loader import data_load
import tensorflow as tf
from unet_pytorch import build_unet
import torch.nn as nn
import json
import xarray as xr
from torch.utils.tensorboard import SummaryWriter
from copy import deepcopy
import pandas as pd
from affine import Affine
from scipy.sparse import *
import rioxarray

In [ ]:
def to_image(img, mixer, raster_dir):
    f = open(*mixer)
    mixer = json.load(f)
    f.close()
    doubleMatrix = mixer["projection"]["affine"]["doubleMatrix"]
    patchesPerRow = mixer["totalPatches"] / mixer["patchesPerRow"]
    img = np.split(img, patchesPerRow)
    img = [np.hstack(l) for l in img]
    img = np.vstack(img).astype("int8")
    ds = xr.DataArray(img)
    img = []
    ds = ds.rio.write_crs(4326, inplace=True)
    transform = Affine(*doubleMatrix)
    ds.rio.write_transform(transform, inplace=True)
    ds.spatial_ref.GeoTransform
    ds.rio.set_spatial_dims("dim_1", "dim_0")
    ds.rio.to_raster(raster_dir)

In [ ]:
!mkdir npyfiles

In [ ]:
# from state_dict import *
def doPrediction(tf_files, model):
    print(tf_files)
    file_ls = np.append(
        np.arange(0, 70) * int(len(list(tf_files)) / 70), len(list(tf_files))
    )
    patches = 0
    np_ls = []
    _ = 1
    for i in range(len(file_ls) - 1):
        print(i)
        data = data_load(
            tf_files[file_ls[i] : file_ls[i + 1]],
            bands,
            description,
            batch_size=batch_size,
        ).get_pridiction_dataset()
        ls = []
        for batch in data.as_numpy_iterator():
            # print(batch.shape)
            predictions = torch.tensor(batch).to(device)
            with torch.no_grad():
                predictions = model(predictions).cpu().numpy()  # .tolist()
            # print(predictions.shape)
            predictions = predictions.argmax(axis=1).squeeze()[
                :,
                x_buffer : x_buffer + kernel_shape[0],
                y_buffer : y_buffer + kernel_shape[1],
            ]
            print("Writing patch " + str(_ * 64) + "...", flush=True)
            _ = _ + 1
            ls.append(predictions)
        np_ls.append(np.vstack(ls).astype("int8"))
    return np_ls
    print("finished")

In [ ]:
ckpt_path = "/content/drive/MyDrive/RDA/model_and_env/sw_ww_test/ww_56_model/checkpoint.pt"

bands = [
    "blue",
    "green",
    "red",
    "nir",
    "swir1",
    "swir2",
    "ndvi",
    "nirv"
]
targets = ["cropland"]
features = bands  # + targets
nclass = 2
batch_size = 32
class_names = ["Others", "Wheat"]
kernel_shape = [256, 256]
kernel_buffer = [128, 128]
# Get set up for prediction.
x_buffer = int(kernel_buffer[0] / 2)
y_buffer = int(kernel_buffer[1] / 2)

buffered_shape = [
    kernel_shape[0] + kernel_buffer[0],
    kernel_shape[1] + kernel_buffer[1],
]
columns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) for k in features
]
description = dict(zip(features, columns))
metrics_dict = {}
device = "cuda"
folder = "prediction"
model = (
    build_unet(len(bands), nclass).cuda()
    if device == "cuda"
    else build_unet(len(bands), nclass)
)
model = nn.DataParallel(model)
model.load_state_dict(torch.load(ckpt_path))
model.eval()

In [ ]:
states = ['North Dakota'] 
for state in states:
    print("predicting ", state)
    folder = (
        "/content/drive/MyDrive/RDA/prediciton/prediction_sw_67_{}".format(
            state
        )
    )
    tf_files = glob(folder + "/*.gz")
    tf_files.sort()
    json_file = glob(folder + "/*.json")
    out_image_file = "/content/drive/MyDrive/RDA/geotiff_result/{}_pytorch.tif".format(
        state
    )
    prediction = doPrediction(tf_files, model)
    # tifname = state
    # files = sorted(glob(f"npyfiles/*.npy"), key=os.path.getmtime)
    img = np.vstack(prediction)
    np.save("/content/drive/MyDrive/RDA/geotiff_result/{}.npy".format(state), img)
    to_image(img, json_file, out_image_file)